## Part 1: cis-eQTL for protein-coding genes

In [43]:
import pandas as pd
import gzip
from pandas_plink import read_plink
import statsmodels.api as sm
import numpy as np

### Step 1: Load and Filter Protein-Coding Genes, Match Sample IDs

In [45]:
# Load gene annotations and gene exp data
gene_annot = pd.read_csv('gene_annot.txt.gz', sep='\t', compression='gzip', header=0)
gene_exp = pd.read_csv('GD462.GeneQuantRPKM.50FN.samplename.resk10.txt.gz', sep='\t', compression='gzip', header=0)

In [46]:
gene_annot.head()

,ID,CHR,START,STOP,SYM,TYPE
0,DDX11L1,1,11868,13052,ENSG00000223972,transcribed_unprocessed_pseudogene
1,OR4F5,1,69090,70008,ENSG00000186092,protein_coding
2,FAM87B,1,817370,819834,ENSG00000177757,lincRNA
3,LINC00115,1,826205,827522,ENSG00000225880,lincRNA
4,LINC01128,1,827607,859446,ENSG00000228794,processed_transcript


In [47]:
gene_exp.head()

,TargetID,Gene_Symbol,Chr,Coord,HG00096,HG00097,HG00099,HG00100,HG00101,HG00102,...,NA20810,NA20811,NA20812,NA20813,NA20814,NA20815,NA20816,NA20819,NA20826,NA20828
0,ENSG00000152931.6,ENSG00000152931.6,5,59783540,0.101858,0.078110,0.048981,0.118597,0.004035,0.010925,...,0.088601,0.240010,0.137175,0.148494,0.038643,0.088509,0.029204,0.024423,0.044816,0.139186
1,ENSG00000183696.9,ENSG00000183696.9,7,48128225,8.183805,5.686911,2.434653,3.830894,6.612288,4.709646,...,13.428205,6.094500,12.536000,2.217262,3.573394,7.583364,4.052882,1.570378,4.900372,6.737308
2,ENSG00000139269.2,ENSG00000139269.2,12,57846106,1.199910,1.573572,0.521616,1.447225,3.565791,1.982681,...,3.225880,1.996067,2.854923,2.267343,1.331201,2.187895,1.004250,3.003316,1.984362,1.684954
3,ENSG00000169129.8,ENSG00000169129.8,10,116164515,0.831940,0.069778,0.931086,0.620941,1.660668,0.570481,...,1.023381,1.127852,0.774409,1.495854,0.895342,1.513521,0.826377,1.021201,0.952502,0.740565
4,ENSG00000134602.11,ENSG00000134602.11,X,131157293,27.646422,24.395572,16.445374,24.806650,25.113349,19.233988,...,25.079490,28.725528,24.450520,27.264069,26.912814,29.509210,26.462331,25.624009,25.707741,22.824957


In [59]:
file_path = "./LDREF"
cis_window = 500000
chrom_num = 22
file_base = f"{file_path}/1000G.EUR.{chrom_num}"  
bim, fam, bed = read_plink(file_base)

gene_exp_df = pd.read_csv('GD462.GeneQuantRPKM.50FN.samplename.resk10.txt.gz', sep='\t', compression='gzip', header=0)

# Find intersection with gene expression data
gene_exp_samples = gene_exp_df.columns[4:]
matched_samples = set(gene_exp_samples).intersection(fam['iid'])

# Filter gene expression data
gene_exp = gene_exp_df[['TargetID', 'Gene_Symbol', 'Chr', 'Coord'] + list(matched_samples)]
gene_exp.loc[:,'Gene_Symbol'] = gene_exp['Gene_Symbol'].apply(lambda x: x.split('.')[0])
gene_annot = pd.read_csv('gene_annot.txt.gz', sep='\t', compression='gzip', header=0)

gene_data = pd.merge(gene_annot, gene_exp, left_on='SYM', right_on='Gene_Symbol').drop(columns=['TargetID', 'Gene_Symbol', 'Chr'])

Mapping files: 100%|██████████| 3/3 [00:00<00:00, 26.02it/s]


In [60]:
gene_data

,ID,CHR,START,STOP,SYM,TYPE,Coord,NA20826,NA20797,HG00271,...,NA12814,NA20542,NA20761,NA12341,HG00102,HG00174,HG00185,NA06984,NA20790,NA12890
0,DDX11L1,1,11868,13052,ENSG00000223972,transcribed_unprocessed_pseudogene,11869,0.405439,0.419416,0.333496,...,0.570705,0.171298,0.337986,0.598353,0.114510,0.291365,0.250563,0.321952,0.404102,0.061793
1,FAM87B,1,817370,819834,ENSG00000177757,lincRNA,752751,0.199947,0.162131,0.218085,...,0.253909,0.058518,0.161440,0.148978,0.030556,0.210119,0.146845,0.192753,0.209997,0.192286
2,LINC00115,1,826205,827522,ENSG00000225880,lincRNA,762902,1.274371,1.406559,1.462647,...,1.370826,0.831591,2.087303,1.428537,0.965335,1.265588,1.378858,2.390641,1.267142,0.985562
3,LINC01128,1,827607,859446,ENSG00000228794,processed_transcript,762988,4.836352,3.165323,13.984298,...,3.439364,4.462475,4.068775,4.108143,6.098974,3.341593,4.263357,11.285223,4.158683,3.193700
4,FAM41C,1,868070,876903,ENSG00000230368,lincRNA,812283,0.123270,0.588116,0.211511,...,0.340717,0.141471,0.298997,0.266485,0.153998,0.124766,0.418450,0.366051,0.174839,0.309491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14450,CHKB-AS1,22,50583025,50583877,ENSG00000205559,antisense,51021455,0.794349,0.516994,0.574955,...,0.534704,0.053769,0.323523,0.557924,0.421676,0.791983,0.292458,0.494860,0.476570,0.105976
14451,ARSA,22,50622753,50628179,ENSG00000100299,protein_coding,51066607,2.780968,6.308448,5.876262,...,7.230366,5.777673,5.857446,4.397727,4.972319,6.550611,5.184058,5.029935,6.540218,6.640773
14452,SHANK3,22,50674414,50733298,ENSG00000251322,protein_coding,51113070,0.011365,-0.001285,0.006871,...,0.003169,0.033616,0.001148,0.002436,0.012147,0.020016,0.005485,0.067898,0.006920,-0.008607
14453,RPL23AP82,22,50756947,50801309,ENSG00000184319,transcribed_unprocessed_pseudogene,51195376,13.143818,15.558162,18.452068,...,14.277360,14.128362,7.551572,14.112582,11.708071,17.879646,15.091987,28.128603,12.815981,8.091989


In [ ]:
# Filter for protein-coding genes
protein_coding_genes = gene_data[gene_data['TYPE'] == 'protein_coding']

'"cis_eQTL and PRS.ipynb"\n# Define cis-window (500Kb up and downstream of the gene body)\nprotein_coding_genes[\'cis_start\'] = protein_coding_genes[\'START\'] - cis_window\nprotein_coding_genes[\'cis_end\'] = protein_coding_genes[\'STOP\'] + cis_window\n\nprotein_coding_genes[\'cis_start\'] = protein_coding_genes[\'cis_start\'].apply(lambda x: 0 if x < 0 else x) # Ensure position is nonnegative\n'

In [84]:
# protein_coding_genes = protein_coding_genes.set_index('ID')
protein_coding_genes

,ID,CHR,START,STOP,SYM,TYPE,Coord,NA20826,NA20797,HG00271,...,NA12814,NA20542,NA20761,NA12341,HG00102,HG00174,HG00185,NA06984,NA20790,NA12890
5,SAMD11,1,924879,943377,ENSG00000187634,protein_coding,860260,4.146011,3.670671,3.052371,...,2.861523,3.211139,1.924338,3.479636,3.797854,3.964000,3.970702,4.517360,3.044510,3.295690
6,NOC2L,1,945056,959309,ENSG00000188976,protein_coding,894689,22.312013,24.303132,21.727519,...,19.544767,18.191547,17.812641,22.539378,19.393836,26.829967,25.936055,24.265438,21.047317,19.296154
7,KLHL17,1,960586,965715,ENSG00000187961,protein_coding,895967,3.212906,4.934419,4.412145,...,3.604611,5.481351,4.605640,3.908390,2.360122,3.592379,3.542003,2.668316,3.571868,3.053417
8,PLEKHN1,1,966496,975108,ENSG00000187583,protein_coding,901877,0.090283,0.262406,0.252842,...,0.516615,0.085359,0.239593,0.251316,0.219318,0.344992,0.242810,0.039149,0.224474,0.148824
9,PERM1,1,976498,982117,ENSG00000187642,protein_coding,917497,0.049808,0.236215,0.141782,...,0.236975,0.075714,0.217711,0.282709,0.085554,0.112509,0.073495,0.121403,0.099568,0.194230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14448,CPT1B,22,50573807,50578455,ENSG00000205560,protein_coding,51017899,7.178084,14.179518,15.643342,...,4.210935,10.511528,9.132602,17.861625,7.765292,12.426572,17.227679,14.353921,4.861119,16.712755
14449,CHKB,22,50580189,50601455,ENSG00000100288,protein_coding,51039884,24.056991,16.846540,20.568357,...,48.897922,18.918445,17.159423,10.913719,19.667829,31.349913,20.928803,24.600511,45.184392,12.754475
14451,ARSA,22,50622753,50628179,ENSG00000100299,protein_coding,51066607,2.780968,6.308448,5.876262,...,7.230366,5.777673,5.857446,4.397727,4.972319,6.550611,5.184058,5.029935,6.540218,6.640773
14452,SHANK3,22,50674414,50733298,ENSG00000251322,protein_coding,51113070,0.011365,-0.001285,0.006871,...,0.003169,0.033616,0.001148,0.002436,0.012147,0.020016,0.005485,0.067898,0.006920,-0.008607


In [ ]:
# Save the common sample IDs to a text file
common_sample_ids_txt = './results/protein_coding_cis_eqtl/common_sample_ids.txt'

# Convert the set of common_sample_ids to a sorted list and save to a text file
with open(common_sample_ids_txt, 'w') as file:
    file.write('\n'.join(sorted(matched_samples)))

print(f"Common sample IDs saved to {common_sample_ids_txt}.")


Common sample IDs saved to ./new/common_sample_ids.txt.


In [87]:
sample_indices = fam[fam['iid'].isin(protein_coding_genes.columns)]['i'].unique().tolist()

### Step 2: cis-eQTL Analysis

In [95]:
from scipy import stats
results = []

for chrom in range(1, 23): 
    print(f"Processing chromosome {chrom}...")
    file_base = f"./LDREF/1000G.EUR.{chrom}"
    bim, fam, bed = read_plink(file_base)
    
    chrom_genes = protein_coding_genes[protein_coding_genes['CHR'] == chrom]
    
    # Create SNP-to-index mapping for genotype retrieval
    snp_to_index = {snp_id: idx for idx, snp_id in enumerate(bim['snp'])}

    for i in range(len(chrom_genes)):
        gene = chrom_genes.iloc[i]
        
        idx, start, stop = gene['ID'], gene['START'] - cis_window, gene['STOP'] + cis_window
        expression = gene.iloc[7:].values.astype(float) # (num_samples, )

        filtered_snps = bim[(bim['pos'] >= start) & (bim['pos'] <= stop)]
        
        snp_indices = filtered_snps['snp'].index.to_list()
   
        filtered_genotypes = bed[snp_indices, :][:, sample_indices] # (num_snp, num_samples)
        
        snp_genotype_mean = np.mean(filtered_genotypes, axis=1, keepdims=True)  # (num_snp, 1)
        expression_mean = np.mean(expression)  # scalar

        centered_genotypes = filtered_genotypes - snp_genotype_mean
        centered_expression = expression - expression_mean

        numerator = np.dot(centered_genotypes, centered_expression)  # (num_snp,)
        denominator = np.sum(centered_genotypes ** 2, axis=1)  # (num_snp,)

        
        slope = numerator / denominator  # (num_snp,)
        intercept = expression_mean - slope * snp_genotype_mean[:, 0]  # (num_snp,)
        
        fitted_values = intercept[:, None] + slope[:, None] * filtered_genotypes # (num_snps, num_samples)
        ss_total = np.sum((expression - expression_mean) ** 2) # scalar
        ss_residual = np.sum((expression[None, :]  - fitted_values) ** 2, axis=1) # (num_snps, )
        r_squared = 1 - (ss_residual / ss_total) # (num_snps, )
        
        dof = len(expression) - 2  
        std_err = np.sqrt((ss_residual / dof) / denominator) # (num_snps, )
        t_values = slope / std_err # (num_snps, )
        p_values = 2 * stats.t.sf(np.abs(t_values), dof) # (num_snps, )
        
        batch_df = pd.DataFrame({
            'chrom': chrom,              
            'gene_id': idx,                    
            'snp': filtered_snps['snp'].values,
            'pos': filtered_snps['pos'].values,
            'ref': filtered_snps['a0'].values,  
            'alt': filtered_snps['a1'].values, 
            'beta': slope,  
            'se': std_err,                             
            'r_squared': r_squared,           
            'p_value': p_values,                        
        })

        results.append(batch_df)


Processing chromosome 1...


Mapping files: 100%|██████████| 3/3 [00:02<00:00,  1.37it/s]


Processing chromosome 2...


Mapping files: 100%|██████████| 3/3 [00:02<00:00,  1.17it/s]


Processing chromosome 3...


...

Mapping files: 100%|██████████| 3/3 [00:00<00:00,  7.61it/s]


Processing chromosome 21...


Mapping files: 100%|██████████| 3/3 [00:00<00:00, 17.93it/s]


Processing chromosome 22...


Mapping files: 100%|██████████| 3/3 [00:00<00:00, 17.54it/s]


In [96]:
# Combine results into a DataFrame
cis_eqtl_df = pd.concat(results, ignore_index=True)
print("cis-eQTL analysis completed.")

cis-eQTL analysis completed.


In [ ]:
bonferroni_threshold = 0.05 / len(cis_eqtl_df)
print(f"Bonferroni threshold: {bonferroni_threshold}")

# Filter significant SNPs
significant_snps_bonferroni = cis_eqtl_df[cis_eqtl_df["p_value"] < bonferroni_threshold]
significant_snps_bonferroni

Bonferroni threshold: 8.086332270951e-09


,chrom,gene_id,snp,pos,ref,alt,beta,se,r_squared,p_value
76860,1,HTR6,rs6426630,19339070,A,G,-0.548467,0.088976,0.099995,1.994389e-09
109713,1,DHDDS,rs3014697,26053712,A,G,-39.614276,4.253449,0.202314,1.548782e-18
110128,1,DHDDS,rs6699237,26925843,T,C,-17.591285,2.588367,0.118987,4.785655e-11
110131,1,DHDDS,rs4970480,26945712,C,T,-17.591285,2.588367,0.118987,4.785655e-11
296922,1,VAV3,rs17017604,107300823,A,C,-0.995406,0.142295,0.125174,1.399651e-11
...,...,...,...,...,...,...,...,...,...,...
6048190,22,GNAZ,rs6003561,23558472,T,C,-1.001731,0.125484,0.157069,2.189829e-14
6052976,22,DERL3,rs7292309,23386647,G,A,-18.308464,2.692347,0.119107,4.672920e-11
6052998,22,DERL3,rs5996464,23439756,T,C,-16.984428,2.717603,0.102503,1.223080e-09
6053002,22,DERL3,rs16997431,23451236,T,G,-16.984428,2.717603,0.102503,1.223080e-09


In [98]:
significant_snps_bonferroni[significant_snps_bonferroni['chrom'] == 22]

,chrom,gene_id,snp,pos,ref,alt,beta,se,r_squared,p_value
6027369,22,PRODH,rs5746516,18615910,G,T,-2.413425,0.357052,0.117848,5.996750e-11
6048190,22,GNAZ,rs6003561,23558472,T,C,-1.001731,0.125484,0.157069,2.189829e-14
6052976,22,DERL3,rs7292309,23386647,G,A,-18.308464,2.692347,0.119107,4.672920e-11
6052998,22,DERL3,rs5996464,23439756,T,C,-16.984428,2.717603,0.102503,1.223080e-09
6053002,22,DERL3,rs16997431,23451236,T,G,-16.984428,2.717603,0.102503,1.223080e-09
6132589,22,RPS19BP1,rs16985820,39764554,T,C,-24.666313,3.633335,0.118759,5.006830e-11


In [ ]:
sorted_df = cis_eqtl_df.sort_values(by='pos')
os.makedirs('protein_coding_cis_eqtl', exist_ok=True)
# Save to a .txt file
sorted_df.to_csv('./results/protein_coding_cis_eqtl/cis_eqtl_sorted.txt', sep='\t', index=False)

In [105]:
cis_eqtl_df[cis_eqtl_df['gene_id'] == 'DERL3']

,chrom,gene_id,snp,pos,ref,alt,beta,se,r_squared,p_value
6052954,22,DERL3,rs11913797,23337705,C,T,-0.126353,0.427403,0.000255,0.767692
6052955,22,DERL3,rs6003445,23338211,T,C,-0.408739,0.374883,0.003464,0.276344
6052956,22,DERL3,rs6003446,23338503,A,G,0.014007,0.502727,0.000002,0.977788
6052957,22,DERL3,rs6003447,23339200,T,C,0.014007,0.502727,0.000002,0.977788
6052958,22,DERL3,rs6003452,23344094,T,G,-0.408739,0.374883,0.003464,0.276344
...,...,...,...,...,...,...,...,...,...,...
6053452,22,DERL3,rs4822466,24312204,G,A,0.138769,0.371434,0.000408,0.708930
6053453,22,DERL3,rs1006771,24314006,G,T,0.237674,0.392315,0.001072,0.545033
6053454,22,DERL3,rs9612528,24314258,A,G,-1.248317,0.611965,0.012020,0.042133
6053455,22,DERL3,rs140289,24336327,T,C,0.272027,0.430620,0.001165,0.527999


In [ ]:
sorted_df = cis_eqtl_df[cis_eqtl_df['gene_id'] == 'DERL3'].sort_values(by='pos')
sorted_df.to_csv('./results/protein_coding_cis_eqtl/cis_eqtl_derl3.txt', sep='\t', index=False)

In [111]:
cis_eqtl_df

,chrom,gene_id,snp,pos,ref,alt,beta,se,r_squared,p_value
0,1,SAMD11,rs3094315,752566,G,A,-0.029492,0.090693,0.000309,0.745238
1,1,SAMD11,rs3131972,752721,A,G,-0.021811,0.090549,0.000170,0.809796
2,1,SAMD11,rs3131969,754182,A,G,-0.029738,0.095795,0.000282,0.756423
3,1,SAMD11,rs1048488,760912,C,T,-0.016639,0.089730,0.000101,0.852995
4,1,SAMD11,rs3115850,761147,T,C,-0.026929,0.089577,0.000264,0.763882
...,...,...,...,...,...,...,...,...,...,...
6183268,22,RABL2B,rs13056621,51181759,A,G,0.129177,0.271066,0.000664,0.633986
6183269,22,RABL2B,rs3865766,51186228,T,C,-0.047282,0.179912,0.000202,0.792856
6183270,22,RABL2B,rs3888396,51211392,C,T,0.044516,0.251161,0.000092,0.859425
6183271,22,RABL2B,rs2238837,51212875,C,A,-0.057186,0.192923,0.000257,0.767092
